# Clustering Multiple Features

**Project Goal**: Build a clustering model for multiple features. 

Specific Goals:

- Determine the highest variance features in the dataset.

In [12]:
import pandas as pd 
import plotly.express as px 
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted

### Prepare Data

Import